<a href="https://colab.research.google.com/github/AlperYildirim1/Phaseformer/blob/main/Phaseformer_dual_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
PhaseDualTransformer v2 (Hybrid) — Training Script
==================================================
A Dual-Stream Decoder:
  1. Sensory Stream (Standard Attention + RoPE)
  2. Relational Stream (Phase Interference + Complex RoPE + Gated)

Configuration (60M Tier):
  - d_model: 512
  - Depth: 11
  - Split: 4 Sensory Heads / 4 Phase Heads
  - Dropout: 0.0
  - Phase Gate: init -5.0 (sigmoid ≈ 0.007, slowly opens)
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import os
import hashlib
from datetime import datetime
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import get_cosine_schedule_with_warmup
from datasets import load_dataset

# ==========================================
# 1. CONFIGURATION (60M HYBRID)
# ==========================================
HF_DATASET    = "Yujivus/fineweb-edu-phaseformer-250M-v2"

VOCAB_SIZE    = 32768
SEQ_LEN       = 512
BATCH_SIZE    = 32
GRAD_ACCUM    = 4       # Effective batch = 128
EPOCHS        = 1
LR            = 6e-4
D_MODEL       = 512
DEPTH         = 11
N_SENSORY     = 4
N_PHASE       = 4
HEAD_DIM      = 64
DROPOUT       = 0.0
DEVICE        = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_float32_matmul_precision("high")

# ==========================================
# 2. ROTARY POSITION EMBEDDINGS
# ==========================================

class RotaryEmbedding(nn.Module):
    """Standard RoPE for the Sensory Stream."""
    def __init__(self, dim, max_period=10000.0):
        super().__init__()
        inv_freq = 1.0 / (max_period ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)

    def forward(self, seq_len, device):
        t = torch.arange(seq_len, device=device).type_as(self.inv_freq)
        freqs = torch.outer(t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        return emb.cos(), emb.sin()

def apply_rotary_pos_emb(q, k, cos, sin):
    """Apply standard RoPE to real-valued Q, K tensors."""
    cos = cos.view(1, 1, *cos.shape)
    sin = sin.view(1, 1, *sin.shape)
    def rotate_half(x):
        x1, x2 = x.chunk(2, dim=-1)
        return torch.cat((-x2, x1), dim=-1)
    return (q * cos) + (rotate_half(q) * sin), (k * cos) + (rotate_half(k) * sin)

# ==========================================
# 3. PARALLEL PHASE ATTENTION
# ==========================================

class ParallelPhaseAttention(nn.Module):
    """
    Batched multi-head phase attention.
    All heads computed in parallel via single projections.
    Q, K constrained to unit circle. Complex RoPE for position.
    """
    def __init__(self, d_model, n_heads, head_dim, dropout=0.0, max_period=10000.0):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = head_dim

        # Real -> Complex projections (2x for real/imag pairs)
        inner_dim = n_heads * head_dim * 2
        self.W_q = nn.Linear(d_model, inner_dim, bias=False)
        self.W_k = nn.Linear(d_model, inner_dim, bias=False)
        self.W_v = nn.Linear(d_model, inner_dim, bias=False)

        # Complex -> Real bridge
        self.out_proj = nn.Linear(inner_dim, n_heads * head_dim, bias=False)

        self.dropout = nn.Dropout(dropout)

        # Complex RoPE: one frequency per complex dimension
        inv_freq = 1.0 / (max_period ** (torch.arange(0, head_dim).float() / head_dim))
        self.register_buffer("inv_freq", inv_freq)

    def forward(self, x):
        B, L, _ = x.shape

        # A. Project real -> complex
        q = torch.view_as_complex(self.W_q(x).view(B, L, self.n_heads, self.head_dim, 2))
        k = torch.view_as_complex(self.W_k(x).view(B, L, self.n_heads, self.head_dim, 2))
        v = torch.view_as_complex(self.W_v(x).view(B, L, self.n_heads, self.head_dim, 2))

        # B. Unit circle constraint on Q, K (phase-only inductive bias)
        q = q / (q.abs() + 1e-6)
        k = k / (k.abs() + 1e-6)

        # C. Complex RoPE: multiply by e^(iωt)
        t = torch.arange(L, device=x.device, dtype=self.inv_freq.dtype)
        angles = torch.outer(t, self.inv_freq)  # [L, head_dim]
        rotors = torch.polar(torch.ones_like(angles), angles).view(1, L, 1, self.head_dim)
        q = q * rotors
        k = k * rotors

        # D. Attention in complex space
        q = q.permute(0, 2, 1, 3)  # [B, H, L, D]
        k = k.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        # Phase-locking scores: real part of hermitian dot product
        scores = (q @ k.transpose(-2, -1).conj()).real / math.sqrt(self.head_dim)

        # Causal mask
        causal_mask = torch.triu(torch.ones(L, L, device=x.device, dtype=torch.bool), diagonal=1)
        scores = scores.masked_fill(causal_mask.unsqueeze(0).unsqueeze(0), float('-inf'))

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        out = attn.to(v.dtype) @ v

        # E. Complex -> Real output
        out = out.transpose(1, 2)  # [B, L, H, D]
        out_flat = torch.view_as_real(out).flatten(2)  # [B, L, H*D*2]

        return self.out_proj(out_flat)

# ==========================================
# 4. DUAL STREAM ATTENTION (Gated Phase)
# ==========================================

class DualStreamAttention(nn.Module):
    """
    Sensory stream: standard real-valued MHA + RoPE
    Phase stream: complex-valued MHA + complex RoPE + learned gate
    """
    def __init__(self, d_model, n_sensory, n_phase, head_dim=64, dropout=0.0):
        super().__init__()
        self.n_sensory = n_sensory
        self.n_phase = n_phase
        self.head_dim = head_dim

        # Stream A: Sensory (Standard)
        if n_sensory > 0:
            self.sensory_qkv = nn.Linear(d_model, 3 * n_sensory * head_dim, bias=False)

        # Stream B: Phase (Relational)
        if n_phase > 0:
            self.phase_attn = ParallelPhaseAttention(d_model, n_phase, head_dim, dropout)
            # Gate: starts near-zero (sigmoid(-5) ≈ 0.007), learns to open
            self.phase_gate = nn.Parameter(torch.full((1, 1, n_phase * head_dim), -5.0))

        # Integration: concat both streams -> d_model
        total_dim = (n_sensory + n_phase) * head_dim
        self.out_proj = nn.Linear(total_dim, d_model, bias=False)

    def forward(self, x, sensory_rope_cos, sensory_rope_sin):
        streams = []

        # 1. Sensory Stream
        if self.n_sensory > 0:
            B, L, _ = x.shape
            qkv = self.sensory_qkv(x).reshape(B, L, 3, self.n_sensory, self.head_dim)
            q, k, v = qkv.unbind(dim=2)

            # 1. Transpose ALL to [Batch, Heads, Length, Head_Dim]
            q = q.transpose(1, 2)
            k = k.transpose(1, 2)
            v = v.transpose(1, 2)

            # 2. Apply Standard RoPE
            q, k = apply_rotary_pos_emb(q, k, sensory_rope_cos, sensory_rope_sin)

            # 3. Flash Attention (causal)
            out_s = F.scaled_dot_product_attention(
                q, k, v,
                is_causal=True,
                dropout_p=0.0
            )
            streams.append(out_s.transpose(1, 2).flatten(2))

        # 2. Phase Stream (Gated)
        if self.n_phase > 0:
            phase_out = self.phase_attn(x)
            phase_out = torch.sigmoid(self.phase_gate) * phase_out
            streams.append(phase_out)

        # 3. Fuse
        return self.out_proj(torch.cat(streams, dim=-1))

# ==========================================
# 5. PHASE DUAL TRANSFORMER
# ==========================================

class PhaseDualTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, depth, n_sensory, n_phase,
                 head_dim=64, dropout=0.0):
        super().__init__()
        self.d_model = d_model

        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.rotary_emb = RotaryEmbedding(head_dim)

        self.layers = nn.ModuleList([
            nn.ModuleDict({
                "norm1": nn.LayerNorm(d_model),
                "attn": DualStreamAttention(d_model, n_sensory, n_phase, head_dim, dropout),
                "norm2": nn.LayerNorm(d_model),
                "mlp": nn.Sequential(
                    nn.Linear(d_model, 4 * d_model),
                    nn.GELU(),
                    nn.Linear(4 * d_model, d_model),
                )
            })
            for _ in range(depth)
        ])

        self.norm_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size, bias=False)
        self.head.weight = self.token_emb.weight  # Weight tying

    def forward(self, input_ids):
        x = self.token_emb(input_ids) * math.sqrt(self.d_model)
        cos, sin = self.rotary_emb(x.shape[1], x.device)

        for l in self.layers:
            x = x + l["attn"](l["norm1"](x), cos, sin)
            x = x + l["mlp"](l["norm2"](x))

        return self.head(self.norm_f(x))

# ==========================================
# 6. OPTIMIZER
# ==========================================

def configure_optimizers(model, learning_rate, weight_decay=0.1, device_type='cuda'):
    """
    Decays: Linear weights, Embeddings.
    Protects: Biases, LayerNorms, Gates.
    """
    decay_params = []
    nodecay_params = []

    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        # Protect: biases, norms, gates
        if param.ndim < 2 or "norm" in name or "bias" in name or "gate" in name:
            nodecay_params.append(param)
        else:
            decay_params.append(param)

    optim_groups = [
        {'params': decay_params, 'weight_decay': weight_decay},
        {'params': nodecay_params, 'weight_decay': 0.0}
    ]

    decay_count = sum(p.numel() for p in decay_params)
    nodecay_count = sum(p.numel() for p in nodecay_params)
    print(f"🔧 Optimizer: Decay {decay_count/1e6:.1f}M | Protect {nodecay_count/1e3:.1f}K")

    use_fused = (device_type == 'cuda') and \
                ('fused' in torch.optim.AdamW.__init__.__code__.co_varnames)
    extra_args = dict(fused=True) if use_fused else dict()

    return torch.optim.AdamW(optim_groups, lr=learning_rate, **extra_args)

# ==========================================
# 7. TRAINING LOOP
# ==========================================

def collate_causal_lm(batch):
    input_ids = torch.stack([torch.tensor(item["input_ids"], dtype=torch.long) for item in batch])
    return {
        "input_ids": input_ids[:, :-1],
        "labels":    input_ids[:, 1:],
    }

def run_training():
    run_id = hashlib.md5(datetime.now().strftime("%Y%m%d%H%M%S").encode()).hexdigest()[:8]
    SAVE_DIR = os.path.join("/content/drive/My Drive/PhaseFormer", f"Hybrid_60M_{run_id}")
    os.makedirs(SAVE_DIR, exist_ok=True)
    writer = SummaryWriter(log_dir=SAVE_DIR)

    # ---- Data ----
    print(f"⬇️ Loading dataset from {HF_DATASET}...")
    dataset = load_dataset(HF_DATASET)
    train_ds = dataset["train"]
    valid_ds = dataset["validation"]
    print(f"   Train: {len(train_ds):,} chunks | Val: {len(valid_ds):,} chunks")

    train_loader = DataLoader(
        train_ds, batch_size=BATCH_SIZE, shuffle=False,
        collate_fn=collate_causal_lm, num_workers=2, pin_memory=True
    )
    valid_loader = DataLoader(
        valid_ds, batch_size=BATCH_SIZE,
        collate_fn=collate_causal_lm, num_workers=2, pin_memory=True
    )

    # ---- Model ----
    print(f"\n⚡ HYBRID 60M (d={D_MODEL}, L={DEPTH}, S={N_SENSORY}, P={N_PHASE})")
    model = PhaseDualTransformer(
        VOCAB_SIZE, D_MODEL, DEPTH, N_SENSORY, N_PHASE, HEAD_DIM, DROPOUT
    ).to(DEVICE)

    # --- ADD THIS INITIALIZATION BLOCK ---
    def _init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Embedding):
            torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)

    model.apply(_init_weights)
    # -------------------------------------

    total_params = sum(p.numel() for p in model.parameters())
    # Robust unique count (handles weight tying)
    seen = set()
    unique_params = sum(p.numel() for p in model.parameters()
                        if p.requires_grad and id(p) not in seen and not seen.add(id(p)))
    print(f"   Total: {total_params/1e6:.2f}M | Unique: {unique_params/1e6:.2f}M")

    # ---- Optimizer ----
    optimizer = configure_optimizers(model, LR, weight_decay=0.1, device_type="cuda")

    num_update_steps_per_epoch = len(train_loader) // GRAD_ACCUM
    total_steps = num_update_steps_per_epoch * EPOCHS
    warmup_steps = int(total_steps * 0.1)

    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    criterion = nn.CrossEntropyLoss()

    print(f"\n🚀 TRAINING (Run: {run_id})")
    print(f"   Steps: {total_steps:,} | Warmup: {warmup_steps} | LR: {LR}")

    global_step = 0
    best_loss = float('inf')

    for epoch in range(EPOCHS):
        model.train()
        pbar = tqdm(train_loader, desc=f"Ep {epoch+1}/{EPOCHS}")

        for step, batch in enumerate(pbar):
            x = batch['input_ids'].to(DEVICE)
            y = batch['labels'].to(DEVICE)

            logits = model(x)
            loss = criterion(logits.view(-1, VOCAB_SIZE), y.view(-1)) / GRAD_ACCUM
            loss.backward()

            if (step + 1) % GRAD_ACCUM == 0:
                grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1

                train_loss = loss.item() * GRAD_ACCUM
                writer.add_scalar('Loss/Train', train_loss, global_step)
                writer.add_scalar('GradNorm', grad_norm.item(), global_step)
                writer.add_scalar('LR', scheduler.get_last_lr()[0], global_step)
                pbar.set_postfix({
                    'loss': f"{train_loss:.4f}",
                    'gnorm': f"{grad_norm.item():.4f}",
                    'lr': f"{scheduler.get_last_lr()[0]:.2e}"
                })

            # Validation every 250 steps
            if global_step > 0 and global_step % 250 == 0 and (step + 1) % GRAD_ACCUM == 0:
                model.eval()
                val_loss = 0
                val_steps = 0
                with torch.no_grad():
                    for vb in valid_loader:
                        vx = vb['input_ids'].to(DEVICE)
                        vy = vb['labels'].to(DEVICE)
                        val_loss += criterion(model(vx).view(-1, VOCAB_SIZE), vy.view(-1)).item()
                        val_steps += 1
                        if val_steps >= 500:
                            break

                avg_val = val_loss / val_steps
                ppl = math.exp(avg_val) if avg_val < 20 else float('inf')
                writer.add_scalar('Loss/Val', avg_val, global_step)
                writer.add_scalar('PPL/Val', ppl, global_step)
                print(f"\n   ✨ Step {global_step} | Val Loss: {avg_val:.4f} | PPL: {ppl:.2f}")

                if avg_val < best_loss:
                    best_loss = avg_val
                    torch.save(model.state_dict(), os.path.join(SAVE_DIR, "best.pt"))
                    print(f"   💾 Saved best (loss={best_loss:.4f})")

                model.train()

        # End-of-epoch checkpoint
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_loss': best_loss,
            'global_step': global_step,
        }, os.path.join(SAVE_DIR, f"epoch_{epoch+1}.pt"))

    # Final validation
    model.eval()
    val_loss = 0
    val_steps = 0
    with torch.no_grad():
        for vb in valid_loader:
            vx = vb['input_ids'].to(DEVICE)
            vy = vb['labels'].to(DEVICE)
            val_loss += criterion(model(vx).view(-1, VOCAB_SIZE), vy.view(-1)).item()
            val_steps += 1

    avg_val = val_loss / val_steps
    ppl = math.exp(avg_val) if avg_val < 20 else float('inf')
    print(f"\n📊 Final | Val Loss: {avg_val:.4f} | PPL: {ppl:.2f}")

    if avg_val < best_loss:
        best_loss = avg_val
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, "best.pt"))

    writer.close()
    print(f"🎉 Done. Best Loss: {best_loss:.4f}")
    return model

if __name__ == "__main__":
    trained_model = run_training()
    from google.colab import runtime
    runtime.unassign()